## Extract flood zone class for each candidate site
Here we want to identify the flood classification for each of our candidate sites. To do this, we need flood hazard data, which ESRI provides as a [on-line Imagery Layer](https://www.arcgis.com/home/item.html?id=11955f1b47ec41a3af86650824e0c634). 

We've not worked with imagery layers before. These are *raster* datasets, meaning the data are represented not as points, lines, polygons, but as grid of regularly spaced pixel values. So, our task is to determine the value of the pixel located at each exit. Because the image layer lives on-line, we'll use the ArcGIS Python API package (instead of GeoPandas).  

In [ ]:
#Import the necessary packages and components 
from arcgis import GIS
from arcgis.features import FeatureLayer, GeoAccessor

There is one "catch" in using the Flood Risk layer mentioned above: it is "subscriber content" meaning, we need to show that we have an official ArcGIS Online account. In previous notebooks, we've created our link to ArcGIS online with the simple `gis = GIS()` command. However, if we supply that command with the web address of our AGOL portal and our user name, we'll **authenticate** our "gis" object, thus allowing us access to susbcriber content.

Modify the command below to use your own log in (e.g. change `jpfay_790` to use your own NetID). On running the code cell, you will be promted for your password...

In [ ]:
#Create the connection to AGOL
gis = GIS('https://NSOE.maps.arcgis.com','jpfay_790')
print("Logged in as: " + gis.properties.user.username)

Next we have to bring the Flood Hazard image layer into our coding environment. Starting from scratch here would be the steps:
* Open up a generic ArcGIS Online page: https://arcgis.com
* Search for the data you want, i.e. enter `USA Flood Hazard Areas` in the search area.
 * You can filter the results to show just **Layers**, then **Imagery Layers**...
* Eventually you should find the resource we want:<br>https://www.arcgis.com/home/item.html?id=11955f1b47ec41a3af86650824e0c634
* Note the URL of the resource: it contains the resource's unique ID: `11955f1b47ec41a3af86650824e0c634`
* Use that ID to access the data via the `gis.content.get()` command.

In [ ]:
#Get the flood hazard image layer service
flood_layer_service = gis.content.get('11955f1b47ec41a3af86650824e0c634')
flood_layer_service.type

The Image Service contains one or more image layers. It's these image layers that contain the actual data we want. So let's examine what image layers are hosted in this image service and then grab the one we want.

In [ ]:
#Reveal the image layers in the image service
flood_layer_service.layers

The result of the above is a list with just one image layer object. Let's extract that out to its own object.

In [ ]:
#Extract the one (and only) image layer from the service
flood_layer = flood_layer_service.layers[0]

In [ ]:
#Quick preview
flood_layer

Ok, so now we have access to our flood data. The next step is to pull in the MJB&A exits data...

In [ ]:
#Import the exit locations
theExits = GeoAccessor.from_featureclass('../Data/MJBA/Exits.shp')
type(theExits)

The remainder of the analysis involves using the `identify()` function of the image layer ([help](https://developers.arcgis.com/python/api-reference/arcgis.raster.toc.html#arcgis.raster.ImageryLayer.identify)). Briefly, this function identifies the pixel value of a raster layer at the location of a provided point. 

Before applying this function for all points, let's examine how this works for a single point.

In [ ]:
#Extract the first point from our exits spatial dataframe
thePoint = theExits.loc[0,'SHAPE']

In [ ]:
#Identify the pixel value at this point location
theResult = flood_layer.identify(thePoint,return_catalog_items=False)
theResult

We see the tool returns a dictionary, with the value we want associated with the `value` key. Let's extract that value to its own object.

In [ ]:
#Extract the value to its own object
theValue = theResult['value']

So, we see that for the first point in our Exits dataset, the "value" is `8`. What does **8** mean? To answer that, we'll have to dig into the attribute table of the imagery layer dataset. 

We do this with the [attribute_table()]() function, which returns a dictionary of attributes for the imagery layer. Within this dictionary is an item called `features` which itself returns a list of the different raster values each pixel can have. 

In [ ]:
#Get a list of the raster feature attributes in the imagery layer
theAttributes = flood_layer.attribute_table()['features']
len(theAttributes)

In [ ]:
#View the first of these "attributes"
theAttributes[0]

We see that each item in `theAttributes` is a Python dictionary, and the value we want in these dictionaries corresponds, in our case, to the `ClassName` key. So, we want to generate a list of all the ClassName values...

In [ ]:
#Create an empty list to hold the class values
classNames = []
#Iterate through all the Attribute values, extract the ClassValue, and add to the list
for theAttribute in theAttributes:
    className = theAttribute['attributes']['ClassName']
    classNames.append(className)
classNames

→ Something called Python list comprehension allows us to do this much more efficiently:

In [ ]:
classNames = [theAttribute['attributes']['ClassName'] for theAttribute in theAttributes]
classNames

So, we see that out value of **8** corresponds to the 8th item in the list. To view this programmatically, we need to convert our `theValue` to an integer and subtract 1 because Python list indices start at zero. 

In [ ]:
#Reveal the actual class value associated with our point
classNames[int(theValue) - 1]

Our first point falls in an "`Area of Minimal Flood Hazard`"!

Putting all this together, we can define a Python function that returns the flood class value for a supplied point:

In [ ]:
def getFloodClass(thePoint):
    return classNames[int(flood_layer.identify(thePoint)['value'])-1]

And then we can apply this function to all items in the "SHAPE" field of our spatial dataframe (this should take a few moments as it's pinging ESRIs server for each exit):

In [ ]:
#Apply the "getFloodClass()" function to each point
theExits['Flood_Zone'] = theExits['SHAPE'].apply(lambda x: getFloodClass(x))

In [ ]:
#Have a look at our result
theExits.head()

Finally, let's save our output as a CSV and a Shapefile...

In [ ]:
#Save our modified dataframe to a csv
theExits.to_csv('../Data/exits_with_flood.csv')

In [ ]:
#Save out modified dataframe to a shapefile
theExits.spatial.to_featureclass('../Data/exits_flood.shp')